### Using Scispacy

In [1]:
### chose to do this from cli
#%pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz

In [1]:
import pandas as pd

In [3]:
a = "hpi 17yo presents palpitations patient reports 3-4 months intermittent episodes heart beating/pounding chest 2 days ago soccer game episode time chest pressure felt going pass lose conciousness note patient endorses abusing adderall primarily study 1-3 times per week recent soccer game took adderrall night morning game denies shortness breath diaphoresis fevers chills headache fatigue changes sleep changes vision/hearing abdominal paun"
a = pd.DataFrame(a.split(), columns=["this"])
a.this.value_counts()

game                3
soccer              2
patient             2
changes             2
chest               2
hpi                 1
study               1
1-3                 1
times               1
per                 1
week                1
recent              1
took                1
adderrall           1
night               1
morning             1
shortness           1
denies              1
adderall            1
breath              1
diaphoresis         1
fevers              1
chills              1
headache            1
fatigue             1
sleep               1
vision/hearing      1
abdominal           1
primarily           1
endorses            1
abusing             1
17yo                1
presents            1
palpitations        1
reports             1
3-4                 1
months              1
intermittent        1
episodes            1
heart               1
beating/pounding    1
2                   1
days                1
ago                 1
episode             1
time      

In [2]:
patient_notes = pd.read_csv("patient_notes.csv")
features = pd.read_csv("features.csv")

print(patient_notes[patient_notes["case_num"] == 0][:10])
features[features["case_num"] == 0][:10]

   pn_num  case_num                                         pn_history
0       0         0  17-year-old male, has come to the student heal...
1       1         0  17 yo male with recurrent palpitations for the...
2       2         0  Dillon Cleveland is a 17 y.o. male patient wit...
3       3         0  a 17 yo m c/o palpitation started 3 mos ago; \...
4       4         0  17yo male with no pmh here for evaluation of p...
5       5         0  17 yo m, presenting with palpitations/increase...
6       6         0  17 yo male presents with 3-4 month history of ...
7       7         0  17 yo male presents with a 2-3 month history o...
8       8         0  HPI: 17 YO complains of intermittent heart pal...
9       9         0  Mr. Cleveland is a 17 yo M who presents to cli...


,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myoc...
1,1,0,Family-history-of-thyroid-disorder
2,2,0,Chest-pressure
3,3,0,Intermittent-symptoms
4,4,0,Lightheaded
5,5,0,No-hair-changes-OR-no-nail-changes-OR-no-tempe...
6,6,0,Adderall-use
7,7,0,Shortness-of-breath
8,8,0,Caffeine-use
9,9,0,heart-pounding-OR-heart-racing


In [27]:
case_num_zero = patient_notes["pn_history"][patient_notes["case_num"] == 0][:1500]
whole_case_num_zero = "\n".join(case_num_zero.to_list())
len(whole_case_num_zero)
whole_case_num_zero[:1000]

"17-year-old male, has come to the student health clinic complaining of heart pounding. Mr. Cleveland's mother has given verbal consent for a history, physical examination, and treatment\r\n-began 2-3 months ago,sudden,intermittent for 2 days(lasting 3-4 min),worsening,non-allev/aggrav\r\n-associated with dispnea on exersion and rest,stressed out about school\r\n-reports fe feels like his heart is jumping out of his chest\r\n-ros:denies chest pain,dyaphoresis,wt loss,chills,fever,nausea,vomiting,pedal edeam\r\n-pmh:non,meds :aderol (from a friend),nkda\r\n-fh:father had MI recently,mother has thyroid dz\r\n-sh:non-smoker,mariguana 5-6 months ago,3 beers on the weekend, basketball at school\r\n-sh:no std\n17 yo male with recurrent palpitations for the past 3 mo lasting about 3 - 4 min, it happened about 5 - 6 times since the beginning. One time durign a baskeball game two days ago light headedness, pressure in the chest, catching breath, but no fainting. During teh episodes no sweating.

In [28]:
import prepare_jag
clean_note = prepare_jag.basic_clean3(whole_case_num_zero)
clean_note = prepare_jag.remove_stopwords(clean_note, exclude_words=["no"])
len(clean_note)

968544

### run this always

In [29]:
import scispacy
import spacy

from scispacy.linking import EntityLinker

sci_nlp = spacy.load("en_ner_bc5cdr_md")
# This line takes a while, because we have to download ~1GB of data
# and load a large JSON file (the knowledge base). Be patient!
# Thankfully it should be faster after the first time you use it, because
# the downloads are cached.
print("works, yay!")

works, yay!


### From https://gbnegrini.com/post/biomedical-text-nlp-scispacy-named-entity-recognition-medical-records/

In [ ]:
doc = sci_nlp(clean_note)
print("TEXT", "START", "END", "ENTITY TYPE")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

- case num 0 patient notes

In [30]:
case_num_zero_notes = patient_notes[patient_notes["case_num"] == 0]
case_num_zero_notes.head(3)

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...


- case num 0 feature text

In [31]:
case_num_zero_features = features[features["case_num"] == 0]
case_num_zero_features.head(3)

,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myoc...
1,1,0,Family-history-of-thyroid-disorder
2,2,0,Chest-pressure


### merge of the notes and features

In [4]:
# notes_features = patient_notes.merge(right=features, left_on="case_num", right_on="case_num")#.drop("case_num_y", axis=1).rename({"case_num_x": "case_num"}, axis=1)
# print(notes_features.shape)
# notes_features.head()

### Create function to repeat this for each note in the dataframe

In [33]:
def get_doc_ents(clean_text):
    
    sci_doc = sci_nlp(clean_text)
    # print("TEXT", "START", "END", "ENTITY TYPE")
    return {ent.text: [ent.start_char, ent.end_char] for ent in sci_doc.ents}

### the below uses the 626k corpus

In [34]:
# notes_features["clean_pn_history"] = notes_features["pn_history"].apply(prepare_jag.basic_clean3).apply(prepare_jag.remove_stopwords, exclude_words=["no"])

# notes_features["clean_notes_history_entities"] = notes_features["clean_pn_history"].apply(get_doc_ents)

In [35]:
notes_features.head()

,pn_num,case_num,pn_history,feature_num,feature_text
0,0,0,"17-year-old male, has come to the student heal...",0,Family-history-of-MI-OR-Family-history-of-myoc...
1,0,0,"17-year-old male, has come to the student heal...",1,Family-history-of-thyroid-disorder
2,0,0,"17-year-old male, has come to the student heal...",2,Chest-pressure
3,0,0,"17-year-old male, has come to the student heal...",3,Intermittent-symptoms
4,0,0,"17-year-old male, has come to the student heal...",4,Lightheaded


In [13]:
# notes_features["clean_notes_history_entities"].shape

In [14]:
# notes_features["clean_notes_history_entities"].head(50)

In [15]:
case_num_feature_text = {}
for i in range(10):
    case_num_feature_text[i] = features["feature_text"][features["case_num"] == i].to_list()
case_num_feature_text

{0: ['Family-history-of-MI-OR-Family-history-of-myocardial-infarction',
  'Family-history-of-thyroid-disorder',
  'Chest-pressure',
  'Intermittent-symptoms',
  'Lightheaded',
  'No-hair-changes-OR-no-nail-changes-OR-no-temperature-intolerance',
  'Adderall-use',
  'Shortness-of-breath',
  'Caffeine-use',
  'heart-pounding-OR-heart-racing',
  'Few-months-duration',
  '17-year',
  'Male'],
 1: ['No-vaginal-discharge',
  'Weight-loss',
  'Not-sexually-active',
  'Prior-episodes-of-diarrhea',
  '20-year',
  'No-bloody-bowel-movements',
  'Recurrent-bouts-over-past-6-months',
  'Right-sided-LQ-abdominal-pain-OR-Right-lower-quadrant-abdominal-pain',
  'No-urinary-symptoms',
  'Diminished-appetite',
  'Normal-LMP-2-weeks-ago-OR-Normal-last-menstrual-period-2-weeks-ago',
  '8-to-10-hours-of-acute-pain',
  'Female'],
 2: ['Prior-normal-periods',
  'Last-Pap-smear-I-year-ago',
  'IUD',
  'Sexually-active',
  'Vaginal-dryness',
  'Irregular-menses',
  'Recent-nausea-vomiting-OR-Recent-flulike-sy

In [16]:
features[["case_num", "feature_num", "feature_text"]]

,case_num,feature_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myoc...
1,0,1,Family-history-of-thyroid-disorder
2,0,2,Chest-pressure
3,0,3,Intermittent-symptoms
4,0,4,Lightheaded
...,...,...,...
138,9,912,Family-history-of-migraines
139,9,913,Female
140,9,914,Photophobia
141,9,915,No-known-illness-contacts


### creating dictionary of dictionaries
- first dictionary key is the case_num
- first dictionary value (second dictionary key) is the feature_num
- second dictionary value is the feature_text

In [5]:
# looking at feature_num per case_num
for i in features["case_num"].unique():
    print(i, features["feature_num"][features["case_num"] == i].values)

0 [ 0  1  2  3  4  5  6  7  8  9 10 11 12]
1 [100 101 102 103 104 105 106 107 108 109 110 111 112]
2 [200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216]
3 [300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315]
4 [400 401 402 403 404 405 406 407 408 409]
5 [500 501 502 503 504 505 506 507 508 509 510 511 512 513 514 515 516 517]
6 [600 601 602 603 604 605 606 607 608 609 610 611]
7 [700 701 702 703 704 705 706 707 708]
8 [800 801 802 803 804 805 806 807 808 809 810 811 812 813 814 815 816 817]
9 [900 901 902 903 904 905 906 907 908 909 910 911 912 913 914 915 916]


In [6]:
# testing how to get only strings
for a in features["feature_num"][features["case_num"] == 0]:
    print(features["feature_text"][features["feature_num"] == a].to_string()[5:].strip())

Family-history-of-MI-OR-Family-history-of-myoc...
Family-history-of-thyroid-disorder
Chest-pressure
Intermittent-symptoms
Lightheaded
No-hair-changes-OR-no-nail-changes-OR-no-tempe...
Adderall-use
Shortness-of-breath
Caffeine-use
heart-pounding-OR-heart-racing
Few-months-duration
17-year
Male


In [7]:
d = {}
for i in features["case_num"].unique():
    d[i] = {a: features["feature_text"][features["feature_num"] == a].to_string()[5:].strip() for a in features["feature_num"][features["case_num"] == i]}
d

{0: {0: 'Family-history-of-MI-OR-Family-history-of-myoc...',
  1: 'Family-history-of-thyroid-disorder',
  2: 'Chest-pressure',
  3: 'Intermittent-symptoms',
  4: 'Lightheaded',
  5: 'No-hair-changes-OR-no-nail-changes-OR-no-tempe...',
  6: 'Adderall-use',
  7: 'Shortness-of-breath',
  8: 'Caffeine-use',
  9: 'heart-pounding-OR-heart-racing',
  10: 'Few-months-duration',
  11: '17-year',
  12: 'Male'},
 1: {100: 'No-vaginal-discharge',
  101: 'Weight-loss',
  102: 'Not-sexually-active',
  103: 'Prior-episodes-of-diarrhea',
  104: '20-year',
  105: 'No-bloody-bowel-movements',
  106: 'Recurrent-bouts-over-past-6-months',
  107: 'Right-sided-LQ-abdominal-pain-OR-Right-lower-q...',
  108: 'No-urinary-symptoms',
  109: 'Diminished-appetite',
  110: 'Normal-LMP-2-weeks-ago-OR-Normal-last-menstrua...',
  111: '8-to-10-hours-of-acute-pain',
  112: 'Female'},
 2: {200: 'Prior-normal-periods',
  201: 'Last-Pap-smear-I-year-ago',
  202: 'IUD',
  203: 'Sexually-active',
  204: 'Vaginal-dryness',
 

In [8]:
df = pd.DataFrame(data=[d.keys(), d.values()]).T
df.columns = ["case_num", "features"]
df

,case_num,features
0,0,{0: 'Family-history-of-MI-OR-Family-history-of...
1,1,"{100: 'No-vaginal-discharge', 101: 'Weight-los..."
2,2,"{200: 'Prior-normal-periods', 201: 'Last-Pap-s..."
3,3,{300: 'FHx-of-PUD-OR-Family-history-of-peptic-...
4,4,"{400: 'Lack-of-other-thyroid-symptoms', 401: '..."
5,5,"{500: 'Onset-5-years-ago', 501: 'Female', 502:..."
6,6,"{600: 'Subjective-fevers', 601: 'Male', 602: '..."
7,7,"{700: 'Female', 701: 'Weight-Gain', 702: 'heav..."
8,8,"{800: 'Increased-appetite', 801: 'Son-died-3-w..."
9,9,{900: 'No-relief-with-Motrin-OR-no-relief-with...


In [9]:
notes_features2 = patient_notes.merge(right=df, left_on="case_num", right_on="case_num")
notes_features2

,pn_num,case_num,pn_history,features
0,0,0,"17-year-old male, has come to the student heal...",{0: 'Family-history-of-MI-OR-Family-history-of...
1,1,0,17 yo male with recurrent palpitations for the...,{0: 'Family-history-of-MI-OR-Family-history-of...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...,{0: 'Family-history-of-MI-OR-Family-history-of...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...,{0: 'Family-history-of-MI-OR-Family-history-of...
4,4,0,17yo male with no pmh here for evaluation of p...,{0: 'Family-history-of-MI-OR-Family-history-of...
...,...,...,...,...
42141,95330,9,Ms. Madden is a 20 yo female presenting w/ the...,{900: 'No-relief-with-Motrin-OR-no-relief-with...
42142,95331,9,A 20 YO F CAME COMPLAIN A DULL 8/10 HEADACHE T...,{900: 'No-relief-with-Motrin-OR-no-relief-with...
42143,95332,9,Ms. Madden is a 20yo female who presents with ...,{900: 'No-relief-with-Motrin-OR-no-relief-with...
42144,95333,9,Stephanie madden is a 20 year old woman compla...,{900: 'No-relief-with-Motrin-OR-no-relief-with...


In [41]:
notes_features2.case_num.value_counts()

3    9753
5    6909
4    5405
9    5151
8    4196
7    4101
0    2268
2    1958
6    1597
1     808
Name: case_num, dtype: int64

In [10]:
notes_features2.case_num.unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=object)

### redo cleaning and finding entities

In [42]:
notes_features2["clean_pn_history"] = notes_features2["pn_history"].apply(prepare_jag.basic_clean3).apply(prepare_jag.remove_stopwords, exclude_words=["no"])
notes_features2.head()

,pn_num,case_num,pn_history,features,clean_pn_history
0,0,0,"17-year-old male, has come to the student heal...",{0: 'Family-history-of-MI-OR-Family-history-of...,17-year-old male come student health clinic co...
1,1,0,17 yo male with recurrent palpitations for the...,{0: 'Family-history-of-MI-OR-Family-history-of...,17 yo male recurrent palpitations past 3 mo la...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...,{0: 'Family-history-of-MI-OR-Family-history-of...,dillon cleveland 17 male patient no significan...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...,{0: 'Family-history-of-MI-OR-Family-history-of...,17 yo c/o palpitation started 3 mos ago nothin...
4,4,0,17yo male with no pmh here for evaluation of p...,{0: 'Family-history-of-MI-OR-Family-history-of...,17yo male no pmh evaluation palpitations state...


In [43]:
notes_features2["note_entities"] = notes_features2["clean_pn_history"].apply(get_doc_ents)
notes_features2.head()

,pn_num,case_num,pn_history,features,clean_pn_history,note_entities
0,0,0,"17-year-old male, has come to the student heal...",{0: 'Family-history-of-MI-OR-Family-history-of...,17-year-old male come student health clinic co...,"{'heart pounding': [56, 70], 'dispnea': [257, ..."
1,1,0,17 yo male with recurrent palpitations for the...,{0: 'Family-history-of-MI-OR-Family-history-of...,17 yo male recurrent palpitations past 3 mo la...,"{'palpitations': [21, 33], 'chest catching bre..."
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...,{0: 'Family-history-of-MI-OR-Family-history-of...,dillon cleveland 17 male patient no significan...,"{'complaints heart pounding': [61, 86], 'pain'..."
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...,{0: 'Family-history-of-MI-OR-Family-history-of...,17 yo c/o palpitation started 3 mos ago nothin...,"{'palpitation': [10, 21], 'nausea vomiting hea..."
4,4,0,17yo male with no pmh here for evaluation of p...,{0: 'Family-history-of-MI-OR-Family-history-of...,17yo male no pmh evaluation palpitations state...,"{'palpitations': [28, 40], '1-2 times month pe..."


In [ ]:
num_records_show = int(input("Please select how many records to show. Must be less than 1000, but the smaller the better due to timing and length: "))

case_num_one = notes_features2["pn_history"][notes_features2["case_num"] == 1][:num_records_show]
whole_case_num_one = "\n".join(case_num_one.to_list())

sci_doc = sci_nlp(whole_case_num_one)

In [ ]:
spacy.displacy.render(sci_doc, style='ent', jupyter=True);

### From Scispacy GitHub

In [ ]:
# NOTE: The resolve_abbreviations parameter is optional, and requires that
# the AbbreviationDetector pipe has already been added to the pipeline. Adding
# the AbbreviationDetector pipe and setting resolve_abbreviations to True means
# that linking will only be performed on the long form of abbreviations.
sci_nlp1 = spacy.load("en_ner_bc5cdr_md")
sci_nlp1.add_pipe("scispacy_linker", config={"linker_name": "umls"}) # {"resolve_abbreviations": True,} 

#### with case_num == 1

In [ ]:
sci_doc1 = sci_nlp(whole_case_num_one)
# Let's look at a random entity!
# sci_doc1 comes from above
entity1 = sci_doc1.ents[1]
print(f"Name: {entity1}")

In [ ]:
# Each entity is linked to UMLS with a score
# (currently just char-3gram matching).
linker = sci_nlp.get_pipe("scispacy_linker")
for umls_ent in entity._.kb_ents:
	print(linker.kb.cui_to_entity[umls_ent[0]])

#### with case_num == 0

In [ ]:
case_num_zero = notes_features2["pn_history"][notes_features2["case_num"] == 0][:num_records_show]
whole_case_num_zero = "\n".join(case_num_zero.to_list())


In [ ]:
sci_doc1 = sci_nlp(whole_case_num_zero)
# Let's look at a random entity!
# sci_doc1 comes from above
entity1 = sci_doc1.ents[1]
print(f"Name: {entity1}")

In [ ]:
entities = sci_doc1.ents[0]
linker = sci_nlp.get_pipe("scispacy_linker")
for umls_ent in entities._.kb_ents:
    print(linker.kb.cui_to_entity[umls_ent[0]])

### comparing locations to features

In [ ]:
notes_features2[["case_num", "features", "note_entities"]]

In [ ]:
d_keys = []
for dic in notes_features2["features"].values:
    if dic.keys() not in d_keys:
        d_keys.append(dic.keys())
d_keys

In [ ]:
notes_features2["features"].values

In [ ]:
for dic in range(len(notes_features2["features"])):
    for key in notes_features2["features"][dic].keys():
        if notes_features2["features"][dic][key] in notes_features2["note_entities"][dic]:
            print(True)